In [1]:
from scipy.ndimage import rotate, shift
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from sklearn.linear_model import LinearRegression
import scipy.stats as sp
import pandas as pd
import statsmodels.api as sm
import math
import cmath

In [2]:
#INPUT

x_min = -20
x_max = 100
y_min = -50
y_max = 50
z_min = 0
z_max = 30

step_y = 1                                                   

stability = 1 #set from 1-6                              

measured_x = 63                                                   
measured_y = 38                                                         
measured_z = 0                                                          

stack_x = 34
stack_y = 13
stack_height = 4       
emission_rate = 1                                                                     
windspeed=4.7
wind_direction=90


In [3]:
#GRID & STEPSIZE CALCULATION 

stability_str = ['Very unstable', 'Moderately unstable', 'Slightly unstable', 'Neutral', 'Moderately stable', 'Very stable'] # Possibly Useless
ssv= stability_str[stability] # Possibly Useless
dx=((x_max - x_min)/(y_max - y_min))* step_y
dz=((z_max - z_min)/(y_max - y_min))* step_y
x_range=np.arange(x_min-(0.5*x_max),(x_max+(0.5*x_max)),dx)
y_range=np.arange(y_min-(0.5*y_max),(y_max+(0.5*y_max))+1,step_y)
z_range=np.arange(z_min,z_max+dz,dz)
X,Y,Z = np.meshgrid(x_range,y_range,z_range)

In [4]:
#WIND RELATED CALCULATIONS

height_slice_direction= math.atan2(measured_y - stack_y, measured_x - stack_x) *(180/(math.pi))
x_origin=X-stack_x
y_origin=Y-stack_y

wind_xcomponent=windspeed*math.sin(math.radians(wind_direction-180))
wind_ycomponent=windspeed*math.cos(math.radians(wind_direction-180))

dot_product=wind_xcomponent*x_origin+wind_ycomponent*y_origin
magnitudes= windspeed*((x_origin**2)+(y_origin**2))**0.5

subtended=np.arccos(dot_product/magnitudes)
hypotenuse=((x_origin**2)+(y_origin**2))**0.5
downwind=np.cos(subtended)*hypotenuse
downwindimag=np.vectorize(complex)(downwind.real, downwind.imag)

In [5]:
#PASQUILL CONSTANTS & SIGMA CALCULATION

stability_class= {1 : (122.8,0.94470,24.1670,2.5334),
                  2: (90.673, 0.93198, 18.3330, 1.8096),
                  3: (61.141, 0.91465, 12.5, 1.0857),
                  4: (34.459, 0.86974, 8.3330, 0.72382),
                  5: (24.26, 0.83660, 6.25, 0.54287),
                  6: (15.209, 0.81558, 4.1667, 0.36191)
                 }
P_a=stability_class[stability][0] 
P_b=stability_class[stability][1]
P_c=stability_class[stability][2]
P_d=stability_class[stability][3]

sig_z = P_a*(abs((X/1000))**P_b)
sig_z[sig_z > 5000] = 5000
theta=0.017453293*(P_c-P_d*np.log((downwindimag/1000)))
sig_y=(465.11628*downwind/1000)*np.tan(theta)

In [20]:
#CROSSWIND AND GAUSSIAN FORMULA

crosswind=np.sin(subtended)*hypotenuse
indices3D=np.where(downwind > 0)
indices2D=np.where((Z==0) & (downwind > 0))
Concentration =emission_rate/(2*math.pi*windspeed*sig_y[indices2D]*sig_z[indices2D])*math.e**(-crosswind[indices2D]**2/(2*sig_y[indices2D]**2))* (math.e**(-(Z[indices2D]-stack_height)**2/(2*sig_z[indices2D]**2))+ math.e**(-(Z[indices2D]+stack_height)**2/(2*sig_z[indices2D]**2)) )
#Concentration =emission_rate/(2*math.pi*windspeed*sig_y[indices]*sig_z[indices])*math.e**(-crosswind[indices]**2/(2*sig_y[indices]**2))* (math.e**(-(-stack_height)**2/(2*sig_z[indices]**2))+ math.e**(-(stack_height)**2/(2*sig_z[indices]**2)) )

Concentration=np.real(Concentration)
print(Concentration,np.shape(Concentration),sig_y)


[1.51338041e-06 1.39522250e-06 1.28214094e-06 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00] (13137,) [[[ 27.81631978+0.j          27.81631978+0.j
    27.81631978+0.j         ...  27.81631978+0.j
    27.81631978+0.j          27.81631978+0.j        ]
  [ 27.5280213 +0.j          27.5280213 +0.j
    27.5280213 +0.j         ...  27.5280213 +0.j
    27.5280213 +0.j          27.5280213 +0.j        ]
  [ 27.23935877+0.j          27.23935877+0.j
    27.23935877+0.j         ...  27.23935877+0.j
    27.23935877+0.j          27.23935877+0.j        ]
  ...
  [-29.26201296+9.56946847j -29.26201296+9.56946847j
   -29.26201296+9.56946847j ... -29.26201296+9.56946847j
   -29.26201296+9.56946847j -29.26201296+9.56946847j]
  [-29.54056606+9.6659035j  -29.54056606+9.6659035j
   -29.54056606+9.6659035j  ... -29.54056606+9.6659035j
   -29.54056606+9.6659035j  -29.54056606+9.6659035j ]
  [-29.8188044 +9.76229418j -29.8188044 +9.76229418j
   -29.8188044 +9.76229418j ... -29.8188044 +9.76229418j
   -29.8

In [10]:
print('change')